### Imports

In [1]:
import numpy as np
import pandas as pd

### Read data from logs
Each of the provided logs contains frequencies for each of the 1280 tero-instances, for 100 pufs (100 different board)

In [2]:
def extract_frequencies(csv_file, counter_bits):
    data = pd.read_csv(csv_file, sep=',', skiprows=1, names=["BoardNumber","DataSent", "#Sent", "DataReceived", "#Received"])
    responsesRaw = np.array(data['DataReceived'])
    responses = []
    for responseRaw in responsesRaw:
        response = []
        for i in range(0, len(responseRaw), int(counter_bits/4)):
            response.append(int(responseRaw[i:i+int(counter_bits/4)], 16))
        responses.append(response)
    responses = np.array(responses)
    return np.transpose(responses) # for each row, an instance of tero. for each column, the value in that fpga

In [25]:
freqs1 = extract_frequencies("TERO/log_20161008U235656_Small_ChosenRepetition_ChosenTime_22.0C.csv", 32)
freqs2 = extract_frequencies("TERO/log_20161009U002207_Small_ChosenRepetition_ChosenTime_22.0C.csv", 32)
freqs3 = extract_frequencies("TERO/log_20161009U023930_Small_ChosenRepetition_ChosenTime_-0.0C.csv", 32)
freqs4 = extract_frequencies("TERO/log_20161009U110717_Small_ChosenRepetition_ChosenTime_44.1C.csv", 32)

In [26]:
print(freqs1.shape)
print(freqs1)

(1280, 100)
[[  71870  239361  165805 ...  126003   96193   72745]
 [  61075 1131202  936723 ...   63300  122902  123455]
 [  68815   93162  373604 ...   43073   73202  447935]
 ...
 [  85216  104602   91431 ...  142617  219258  164340]
 [ 163790   66403  118687 ...  133235  124278  148629]
 [ 143384  118096   52582 ...  194886   83166  447925]]


### Group frequencies in batches

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16

[0] 0 4 8 12 16  
[1] 1 5 9 13 17  

[0] 0 8 16  x 160  
[1] 1 9 17  
[2] 2 10 18  
[3] 3 ...  
[4] 4 12 20  
...  
[7]  

In [29]:
def group_frequencies(freqs):
    # Based on the assumption we have 8 different cell types, we group each type in the right column
    NUM_INSTANCES = freqs.shape[0]  # 1280
    NUM_FPGA = freqs.shape[1]   # 100
    NUM_TYPES = 8
    batchArray = np.zeros((int(NUM_INSTANCES/NUM_TYPES),NUM_TYPES,NUM_FPGA))
    for fpga in range(0, NUM_FPGA):
        for instance_type in range(0, NUM_TYPES):
            batchArray[:,instance_type,fpga] = freqs[instance_type::NUM_TYPES,fpga] # from the starting index, we sample every 8 elements
    # batcharray: 160x8x100
    return batchArray

In [30]:
batched1 = group_frequencies(freqs1)
batched2 = group_frequencies(freqs2)
batched3 = group_frequencies(freqs3)
batched4 = group_frequencies(freqs4)

In [31]:
batched1[:,0,0] # first batch, first fpga
batched1.shape

(160, 8, 100)

### 2Comp Neighbour Algorithm
For each batch, the response is given by comparing each frequency of the batch with the next one:

* For Each  i = [1,3,5,...,15]
* Choose    j = i + 1
* If f(i) > f(j) => res = 1, else => res = 0

In this way BATCH_SIZE/2 bits for each batch is extracted (in our case 8 bits per batch).


In [89]:
def alg_2comp_neigh_batch (batch):
    """ This function compares the frequencies of a single batch"""
    BATCH_SIZE = len(batch) # 160
    response = np.zeros((int(BATCH_SIZE/2)), dtype=np.int) # 80
    response = [int(bool_) for bool_ in batch[::2] > batch [1::2] ]
    return response

#def alg_2comp_neigh_batch (batch):
#    """ This function compares the frequencies of a single batch"""
#    BATCH_SIZE = len(batch) # 160
#    response = np.zeros((int(BATCH_SIZE/2))) # 80
#    for i in range(0, BATCH_SIZE-1, 2):
#        response[i // 2] = int(batch[i] > batch[i+1])
#    return response

In [90]:
def alg_2comp_neigh (experiment_data):
    """ This function compares the frequencies for all batches and all fpgas, given a certain experiment"""
    NUM_BATCHES = experiment_data.shape[1]
    NUM_FPGA = experiment_data.shape[2]
    NUM_RESPONSE_BITS = int(experiment_data.shape[0]/2)

    response = np.zeros(( NUM_RESPONSE_BITS,NUM_BATCHES,NUM_FPGA),dtype=int) #8,80,100
    for currFpga in range (0,NUM_FPGA): #num of fpga
        for currBatch in range (0,NUM_BATCHES): #num of batches
            response[:,currBatch,currFpga] = alg_2comp_neigh_batch(experiment_data[:,currBatch,currFpga])
    
    return response # (80 bit, 8 Batches, 100 fpga)

In [92]:
print(alg_2comp_neigh(batched1)[:,0,0])

[0 1 0 0 0 0 0 1 1 1 0 1 1 0 0 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 0
 0 0 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0
 1 1 0 1 0 1]


In [93]:
data = np.array([
    alg_2comp_neigh(batched1),
    alg_2comp_neigh(batched2),
    alg_2comp_neigh(batched3),
    alg_2comp_neigh(batched4)
], dtype=np.int)

In [94]:
data.shape

(4, 80, 8, 100)

In [95]:
print(data[0,:,0,0])
data[0,:,:,0].shape

[0 1 0 0 0 0 0 1 1 1 0 1 1 0 0 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 0
 0 0 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0
 1 1 0 1 0 1]


(80, 8)

### Analysis

#### Uniformity
The uniformity metric (also called randomness by Yu et al. in [53])
determines how uniform the proportion of 0′ 𝑠 and 1′ 𝑠 are in the PUF
response

In [96]:
def calc_uniformity(response):
    return np.sum([response[i] for i in range(0, len(response))]) / len(response)

def worst_uniformity(data):
    max = 0.5   # ideal value
    sum = 0
    PUF_NUM = data.shape[3]
    EXP_NUM = data.shape[0]
    BATCH_NUM = data.shape[2]
    for puf_num in range(0, PUF_NUM):
        for exp_num in range(0, EXP_NUM):
            for batch_num in range(0, BATCH_NUM):
                u = calc_uniformity(data[exp_num,:,batch_num,puf_num])
                sum = sum + u
                if np.abs(u - 0.5) > np.abs(max - 0.5):
                    max = u
                    if u == 0:
                        print('Experiment for 0 uniformity:\n EXP: ',exp_num,'\n BATCH: ',batch_num, '\n PUF: ',puf_num,'\n\n')
    return max, sum / (PUF_NUM*EXP_NUM*BATCH_NUM)

In [97]:
worst_uniformity(data)

(0.6875, 0.4992031250000003)

#### Reliability
It determines how efficiently a PUF can generate the same response
at different operating conditions (ambient temperatures or supply volt-
ages) over a period of time for a given challenge

In [106]:
def hamming_distance(arr1, arr2): 
    distance = 0
    L = len(arr1)
    for i in range(L):
        if arr1[i] != arr2[i]:
            distance += 1
    return distance

def hd_intra(puf_responses):
    return np.sum([
        hamming_distance(puf_responses[0,:], puf_responses[exp_num,:]) / puf_responses.shape[1]
        for exp_num in range(1, puf_responses.shape[0])]) / puf_responses.shape[0]
    
def reliability(puf_responses):
    return 1 - hd_intra(puf_responses)

def stat_reliability(data):
    min = 1 # ideal value
    max = 0
    sum = 0
    for puf_num in range(0, data.shape[3]):
        for batch_num in range(0, data.shape[2]):
            r = reliability(data[:,:,batch_num,puf_num])
            if r < min:
                min = r
            if r > max:
                max = r
            sum += r
    return min, max, sum / (data.shape[3] * data.shape[2])

In [107]:
stat_reliability(data)

(0.934375, 1.0, 0.9749921874999993)

#### Uniqueness
It measures the variation of responses obtained from different chips
for the same set of challenges

In [100]:
def calc_uniqueness(data):
    num_bits = data.shape[0]
    num_devices = data.shape[1]
    return 2/(num_devices*(num_devices-1)) * np.sum([
        np.sum([
            hamming_distance(data[:,i], data[:,j])/num_bits
            for j in range(i+1, num_devices)])  
        for i in range(0, num_devices-1)])

def worst_uniqueness(data):
    n_esperiments = data.shape[0]
    max = 0.5   # ideal value
    for exp_num in range(0, n_esperiments):
        for ij_couple in range(0, data.shape[2]):
            u = calc_uniqueness(data[exp_num,:,ij_couple,:])
            if np.abs(u - 0.5) > np.abs(max - 0.5):
                max = u
    return max

In [101]:
worst_uniqueness(data)

0.5011363636363637

### Some samples

In [102]:
def bit_array_to_string(arr):
    response_char = ''
    for i in range(0, len(arr)):
        response_char = response_char + ('1' if arr[i] else '0')
    return response_char  

def print_ids_per_fpga(data):
    num_batches = data.shape[1] #80
    out = []
    for curr_batch in range(0, num_batches):
        out.append(bit_array_to_string(data[:,curr_batch]))
    return out

In [103]:
# Let's print all the first 10 ids of the first board at 20°C
print_ids_per_fpga(data[0,:,:,0])[0:10]

['01000001110110011110011110011111110100001100101111010101011000010010001000110101',
 '01010111000001111100000000011000000110100110100110110011010111001000101001011111',
 '11101010101100010100100000001111101110010101011110010000000111101011011010000111',
 '01010101011001110100100111100101110001110100111101011111100011111001000101100100',
 '10100001111100011000000101111101010000111101000100111110111001110000011010000011',
 '11110100000001010001010101010111010101110111111010111110010101101110111101010010',
 '11010001000001001001010001000100011100100011101000111110001011011100010100001000',
 '00000001011000111011001001001111101000100000001000001100001110000000010011010110']